# Full Rank Markov and Geographic Rank Markov 

**Author: Wei Kang <weikang9009@gmail.com>**

In [13]:
import libpysal as ps
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
import geopandas as gpd

## Full Rank Markov

In [14]:
from giddy.markov import FullRank_Markov

In [15]:
income_table = pd.read_csv(ps.examples.get_path("usjoin.csv"))
income_table.head()

,Name,STATE_FIPS,1929,1930,1931,1932,1933,1934,1935,1936,...,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,Alabama,1,323,267,224,162,166,211,217,251,...,23471,24467,25161,26065,27665,29097,30634,31988,32819,32274
1,Arizona,4,600,520,429,321,308,362,416,462,...,25578,26232,26469,27106,28753,30671,32552,33470,33445,32077
2,Arkansas,5,310,228,215,157,157,187,207,247,...,22257,23532,23929,25074,26465,27512,29041,31070,31800,31493
3,California,6,991,887,749,580,546,603,660,771,...,32275,32750,32900,33801,35663,37463,40169,41943,42377,40902
4,Colorado,8,634,578,471,354,353,368,444,542,...,32949,34228,33963,34092,35543,37388,39662,41165,41719,40093


### Let's only select per capical incomes in years 1949-2009

In [16]:
pci = income_table[list(map(str,range(1949,2010)))].values
pci

array([[  833,   909,  1045, ..., 31988, 32819, 32274],
       [ 1305,  1367,  1623, ..., 33470, 33445, 32077],
       [  813,   847,   957, ..., 31070, 31800, 31493],
       ...,
       [ 1023,  1056,  1182, ..., 29769, 31265, 31843],
       [ 1387,  1506,  1736, ..., 35839, 36594, 35676],
       [ 1647,  1719,  1955, ..., 43453, 45177, 42504]])

In [17]:
m = FullRank_Markov(pci)
m.ranks

array([[46, 46, 46, ..., 41, 40, 39],
       [25, 26, 23, ..., 36, 38, 41],
       [47, 47, 47, ..., 43, 43, 43],
       ...,
       [40, 42, 42, ..., 47, 46, 42],
       [18, 22, 17, ..., 25, 26, 25],
       [ 7,  9,  8, ...,  6,  6,  7]])

In [18]:
m.transitions

array([[53.,  4.,  2., ...,  0.,  0.,  0.],
       [ 6., 41.,  7., ...,  0.,  0.,  0.],
       [ 1.,  9., 39., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., 31., 13.,  0.],
       [ 0.,  0.,  0., ..., 12., 46.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., 60.]])

Full rank Markov transition probability matrix

In [19]:
m.p

array([[0.88333333, 0.06666667, 0.03333333, ..., 0.        , 0.        ,
        0.        ],
       [0.1       , 0.68333333, 0.11666667, ..., 0.        , 0.        ,
        0.        ],
       [0.01666667, 0.15      , 0.65      , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.51666667, 0.21666667,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.2       , 0.76666667,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [20]:
np.where(m.p == 1)

(array([47]), array([47]))

The probability of staying in rank 48 is $100\%$. Thus, this rank is not communicating with other ranks, meaning that if a US State currently ranks 48th in per capita income, it will always rank 48th in the future. In this case, there is not going to be a unique steady state distribution towards which the Markov Chain converges. Instead, the future distribution will be dependent on whether the system is traped in rank 48. And also because of this, the estimation of the first mean passage time becomes more complicated. Currently, `giddy` does not have a mechanism to deal with it.

The current implementation for estimating the steady state distribution in `giddy` ignores the possibility of being trapped in rank 48 and gives the following distribution.

The estimation of the first mean passage times will incur an error shown as follows:

In [11]:
m.fmpt

array([[1.00000000e+00, 2.00000000e+00, 3.00000000e+00, ...,
        2.50000000e+00, 4.50000000e+00, 2.11669182e+17],
       [7.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        2.00000000e+00, 4.00000000e+00, 2.11669182e+17],
       [9.00000000e+00, 2.00000000e+00, 1.00000000e+00, ...,
        2.00000000e+00, 4.00000000e+00, 2.11669182e+17],
       ...,
       [3.25000000e+01, 2.55000000e+01, 2.45000000e+01, ...,
        1.00000000e+00, 3.00000000e+00, 2.11669182e+17],
       [3.30000000e+01, 2.65000000e+01, 2.55000000e+01, ...,
        1.00000000e+00, 1.00000000e+00, 2.11669182e+17],
       [4.50359963e+15, 4.50359963e+15, 4.50359963e+15, ...,
        4.50359963e+15, 4.50359963e+15, 1.00000000e+00]])

#### At the moment (due to the time limit for submitting the paper), I will suggest to use an external package `quantecon` to check whether the Markov Chain is ergodic. If not, we give some explanation and not calculating or ploting the first mean passage times. 

#### I will refactor `giddy` in the meantime to incorporate the non-ergodic chains when estimating steady state distributions and first mean passage times. 

## Use `quantecon` to check whether the Markov Chain is ergodic and which Markov states are not communicating with others

#### (1) check whether the Markov Chain is ergodic

```python
mc.is_irreducible and mc.is_aperiodic
```

In [31]:
import quantecon as qe
mc = qe.MarkovChain(m.p)

mc.is_irreducible and mc.is_aperiodic

False

In [35]:
mc.stationary_distributions

array([[0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766,
        0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766,
        0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766,
        0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766,
        0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766,
        0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766,
        0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766,
        0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.0212766, 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.     

In [32]:
mc.is_irreducible

False

In [33]:
mc.is_aperiodic

True

#### (2) check which Markov states are not communicating with others
```python
mc.num_communication_classes
```

```python
mc.communication_classes
```

We have two sets of Markov states which are not communicating with each other

In [20]:
mc.num_communication_classes

2

The first set contains all the Markov states (rank 1 to 47) and the second set contains only 1 Markov state (rank 48)

In [19]:
mc.communication_classes

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46]), array([47])]

In [34]:
from giddy.ergodic import steady_state as STEADY_STATE
if mc.num_communication_classes==1:
    STEADY_STATE(mp.)

SyntaxError: unexpected EOF while parsing (<ipython-input-34-c86cad9976ff>, line 3)